##### Copyright 2025 Google LLC.

In [45]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Gemini API: Function calling with Python

<a target="_blank" href="https://colab.research.google.com/github/google-gemini/cookbook/blob/main/quickstarts/Function_calling.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" height=30/></a>

 Function calling lets developers create a description of a function in their code, then pass that description to a language model in a request. The response from the model includes the name of a function that matches the description and the arguments to call it with. Function calling lets you use functions as tools in generative AI applications, and you can define more than one function within a single request.

This notebook provides code examples to help you get started. The documentation's [quickstart](https://ai.google.dev/tutorials/function_calling_python_quickstart) is also a good place to start understanding function calling.

## Setup

### Install dependencies

In [1]:
%pip install -qU 'google-genai'

Note: you may need to restart the kernel to use updated packages.


In [47]:
from google import genai

### Set up your API key

To run the following cell, your API key must be stored it in a Colab Secret named `GOOGLE_API_KEY`. If you don't already have an API key, or you're not sure how to create a Colab Secret, see the [Authentication](../quickstarts/Authentication.ipynb) quickstart for an example.

In [48]:
from google.colab import userdata

GOOGLE_API_KEY = userdata.get("GOOGLE_API_KEY")
client = genai.Client(api_key=GOOGLE_API_KEY)

### Choose a model

Function calling should work with all the [Gemini 2.0](https://ai.google.dev/gemini-api/docs/models/gemini-v2) models with the GenAI SDK. It also works with the 1.5 generation of models.

In [3]:
MODEL_ID="gemini-2.5-flash-preview-04-17" # @param ["gemini-2.5-flash-preview-04-17", "gemini-2.5-pro-preview-03-25", "gemini-2.0-flash", "gemini-2.0-flash-lite"] {"allow-input":true, isTemplate: true}

## Function calling basics

To use function calling, pass a list of functions to the `tools` parameter when creating a [`GenerativeModel`](https://ai.google.dev/api/python/google/generativeai/GenerativeModel). The model uses the function name, docstring, parameters, and parameter type annotations to decide if it needs the function to best answer a prompt.

> Important: The SDK converts function parameter type annotations to a format the API understands (`genai.types.FunctionDeclaration`). The API only supports a limited selection of parameter types, and the Python SDK's automatic conversion only supports a subset of that: `AllowedTypes = int | float | bool | str | list['AllowedTypes'] | dict`

In [4]:
from google.genai import types
def add(a: float, b: float):
    """returns a + b."""
    return a + b


def subtract(a: float, b: float):
    """returns a - b."""
    return a - b


def multiply(a: float, b: float):
    """returns a * b."""
    return a * b


def divide(a: float, b: float):
    """returns a / b."""
    return a / b


operation_tools = [add, subtract, multiply, divide]

## Automatic function calling

Function calls naturally fit in to [multi-turn chats](https://ai.google.dev/gemini-api/docs/text-generation?lang=python#chat) as they capture a back and forth interaction between the user and model. The Python SDK's [`Chat` Session](https://googleapis.github.io/python-genai/index.html#chats) is a great interface for chats because handles the conversation history for you, and using the parameter `automatic_function_calling` (enabled by default) simplifies function calling even further:

In [5]:
chat = client.chats.create(
    model = MODEL_ID,
    config = {
        "tools": operation_tools,
        "automatic_function_calling": {"disable": False} # This line is not needed as automatic_function_calling is enabled by default
    }
)

With automatic function calling enabled, `Chat.send_message` automatically calls your function if the model asks it to.

In the following example, the result appears to simply be a text response containing the correct answer:

In [6]:
response = chat.send_message(
    "I have 57 cats, each owns 44 mittens, how many mittens is that in total?"
)
response.text

'That would be 2508 mittens in total.'

In [7]:
57 * 44

2508

However, by examining the chat history, you can see the flow of the conversation and how function calls are integrated within it.

The `Chat.history` property stores a chronological record of the conversation between the user and the Gemini model. You can get the history using `Chat.get_history()`. Each turn in the conversation is represented by a [`genai.types.Content`](https://googleapis.github.io/python-genai/genai.html#genai.types.Content) object, which contains the following information:

*   **Role**: Identifies whether the content originated from the "user" or the "model".
*   **Parts**: A list of [`genai.types.Part`](https://googleapis.github.io/python-genai/genai.html#genai.types.Part) objects that represent individual components of the message. With a text-only model, these parts can be:
    *   **Text**: Plain text messages.
    *   **Function Call** ([`genai.types.FunctionCall`](https://googleapis.github.io/python-genai/genai.html#genai.types.FunctionCall)): A request from the model to execute a specific function with provided arguments.
    *   **Function Response** ([`genai.types.FunctionResponse`](https://googleapis.github.io/python-genai/genai.html#genai.types.FunctionResponse)): The result returned by the user after executing the requested function.

 In the previous example with the mittens calculation, the history shows the following sequence:

1.  **User**: Asks the question about the total number of mittens.
1.  **Model**: Determines that the multiply function is helpful and sends a FunctionCall request to the user.
1.  **User**: The `Chat` session automatically executes the function (due to `_automatic_function_calling` being set) and sends back a `FunctionResponse` with the calculated result.
1.  **Model**: Uses the function's output to formulate the final answer and presents it as a text response.

In [8]:
from IPython.display import Markdown, display

for content in chat.get_history():
    display(Markdown("###" + content.role + ":"))
    for part in content.parts:
        if part.text:
            display(Markdown(part.text))
        if part.function_call:
            print("Function call: {", part.function_call, "}")
        if part.function_response:
            print("Function response: {", part.function_response, "}")
    print("-" * 80)


###user:

I have 57 cats, each owns 44 mittens, how many mittens is that in total?

--------------------------------------------------------------------------------


###model:

Function call: { id=None args={'a': 57, 'b': 44} name='multiply' }
--------------------------------------------------------------------------------


###user:

Function response: { id=None name='multiply' response={'result': 2508} }
--------------------------------------------------------------------------------


###model:

That would be 2508 mittens in total.

--------------------------------------------------------------------------------


In general the state diagram is:

<img src="https://codelabs.developers.google.com/static/codelabs/gemini-function-calling/img/gemini-function-calling-overview_1440.png" alt="The model can always reply with text, or a FunctionCall. If the model sends a FunctionCall the user must reply with a FunctionResponse" width=50%>

The model can respond with multiple function calls before returning a text response, and function calls come before the text response.

## Manual function calling

For more control, you can process [`genai.types.FunctionCall`](https://googleapis.github.io/python-genai/genai.html#genai.types.FunctionCall) requests from the model yourself. This would be the case if:

- You use a `Chat` with the default `"automatic_function_calling": {"disable": False}`.
- You use [`Client.model.generate_content`](https://googleapis.github.io/python-genai/genai.html#genai.types.) (and manage the chat history yourself).

The following example is a rough equivalent of the [function calling single-turn curl sample](https://ai.google.dev/docs/function_calling#function-calling-single-turn-curl-sample) in Python. It uses functions that return (mock) movie playtime information, possibly from a hypothetical API:

In [9]:
def find_movies(description: str, location: str):
    """find movie titles currently playing in theaters based on any description, genre, title words, etc.

    Args:
        description: Any kind of description including category or genre, title words, attributes, etc.
        location: The city and state, e.g. San Francisco, CA or a zip code e.g. 95616
    """
    return ["Barbie", "Oppenheimer"]


def find_theaters(location: str, movie: str):
    """Find theaters based on location and optionally movie title which are is currently playing in theaters.

    Args:
        location: The city and state, e.g. San Francisco, CA or a zip code e.g. 95616
        movie: Any movie title
    """
    return ["Googleplex 16", "Android Theatre"]


def get_showtimes(location: str, movie: str, theater: str, date: str):
    """
    Find the start times for movies playing in a specific theater.

    Args:
      location: The city and state, e.g. San Francisco, CA or a zip code e.g. 95616
      movie: Any movie title
      thearer: Name of the theater
      date: Date for requested showtime
    """
    return ["10:00", "11:00"]

theater_functions = [find_movies, find_theaters, get_showtimes]

After using `generate_content()` to ask a question, the model requests a `function_call`:

In [10]:
response = client.models.generate_content(
    model=MODEL_ID,
    contents="Which theaters in Mountain View show the Barbie movie?",
    config = {
        "tools": theater_functions,
        "automatic_function_calling": {"disable": True} # This line is not needed as automatic_function_calling is enabled by default
    }
)

response.candidates[0].content.parts

[Part(video_metadata=None, thought=None, code_execution_result=None, executable_code=None, file_data=None, function_call=FunctionCall(id=None, args={'location': 'Mountain View, CA', 'movie': 'Barbie'}, name='find_theaters'), function_response=None, inline_data=None, text=None)]

Since this is not using a `ChatSession` with automatic function calling, you have to call the function yourself.

A very simple way to do this would be with `if` statements:

```python
if function_call.name == 'find_theaters':
  find_theaters(**function_call.args)
elif ...
```

However, since you already made the `functions` dictionary, this can be simplified to:

In [11]:
def call_function(function_call, functions):
    function_name = function_call.name
    function_args = function_call.args
    # Find the function object from the list based on the function name
    for func in functions:
        if func.__name__ == function_name:
            return func(**function_args)

part = response.candidates[0].content.parts[0]

# Check if it's a function call; in real use you'd need to also handle text
# responses as you won't know what the model will respond with.
if part.function_call:
    result = call_function(part.function_call, theater_functions)

print(result)

['Googleplex 16', 'Android Theatre']


Finally, pass the response plus the message history to the next `generate_content()` call to get a final text response from the model. The next code cell is showing on purpose different ways to write down `Content` so you can choose the one that you prefer.

In [12]:
# Build the message history
messages = [
    genai.types.Content(role="user", parts=[genai.types.Part(text="Which theaters in Mountain View show the Barbie movie?.")]),
    genai.types.Content(role="model", parts=[part]),
    {"role":"user", "parts":[{"function_response":{"response":{"output":result}, "name":part.function_call.name}}]},
]

# Generate the next response
response = client.models.generate_content(
    model=MODEL_ID,
    contents=messages,
    config = {
        "tools": theater_functions,
        "automatic_function_calling": {"disable": True}
    }
)
print(response.text)

The Barbie movie is currently playing at the Googleplex 16 and Android Theatre in Mountain View.


## Function calling chain

The model is not limited to one function call, it can chain them until it finds the right answer.

In [13]:
chat = client.chats.create(
    model = MODEL_ID,
    config = {
        "tools": theater_functions,
    }
)

response = chat.send_message(
    "Which comedy movies are shown tonight (01/01/2025) in Mountain view, in which cinema and at what time?"
)

for content in chat.get_history():
    display(Markdown("###" + content.role + ":"))
    for part in content.parts:
        if part.text:
            display(Markdown(part.text))
        if part.function_call:
            print("Function call: {", part.function_call, "}")
        if part.function_response:
            print("Function response: {", part.function_response, "}")
    print("-" * 80)

###user:

Which comedy movies are shown tonight (01/01/2025) in Mountain view, in which cinema and at what time?

--------------------------------------------------------------------------------


###model:

Okay, I can help you with that. First, I need to find the comedy movies playing in Mountain View, CA tonight. Then, for each movie, I will find the theaters showing it and finally get the showtimes for January 1st, 2025.

Let's start by finding the comedy movies.


...

###user:

Function response: { id=None name='get_showtimes' response={'result': ['10:00', '11:00']} }
--------------------------------------------------------------------------------


###model:

Okay, here are the comedy movies the API found playing tonight, January 1st, 2025, in Mountain View, CA, along with their showtimes:

Based on the search, the movies "Barbie" and "Oppenheimer" were found under the "comedy" description.

For "Barbie":
*   At Googleplex 16, the showtimes are 10:00 PM and 11:00 PM.
*   At Android Theatre, the showtimes are 10:00 PM and 11:00 PM.

For "Oppenheimer":
*   At Googleplex 16, the showtimes are 10:00 PM and 11:00 PM.
*   At Android Theatre, the showtimes are 10:00 PM and 11:00 PM.

--------------------------------------------------------------------------------


Here you can see that the model made seven calls to answer your question and used the outputs of them in the subsequent calls and in the final answer.

## Parallel function calls

The Gemini API can call multiple functions in a single turn. This caters for scenarios where there are multiple function calls that can take place independently to complete a task.

First set the tools up. Unlike the movie example above, these functions do not require input from each other to be called so they should be good candidates for parallel calling.

In [15]:
def power_disco_ball(power: bool) -> bool:
    """Powers the spinning disco ball."""
    print(f"Disco ball is {'spinning!' if power else 'stopped.'}")
    return True


def start_music(energetic: bool, loud: bool, bpm: int) -> str:
    """Play some music matching the specified parameters.

    Args:
      energetic: Whether the music is energetic or not.
      loud: Whether the music is loud or not.
      bpm: The beats per minute of the music.

    Returns: The name of the song being played.
    """
    print(f"Starting music! {energetic=} {loud=}, {bpm=}")
    return "Never gonna give you up."


def dim_lights(brightness: float) -> bool:
    """Dim the lights.

    Args:
      brightness: The brightness of the lights, 0.0 is off, 1.0 is full.
    """
    print(f"Lights are now set to {brightness:.0%}")
    return True

house_fns = [power_disco_ball, start_music, dim_lights]

Now call the model with an instruction that could use all of the specified tools.

In [16]:
# Set the chat up with tools.
chat = client.chats.create(
    model = MODEL_ID,
    config = {
        "tools": house_fns,
    }
)

# Call the API
response = chat.send_message(
    "Turn this place into a party!"
)

# Print out each of the function calls requested from this single call.
for content in chat.get_history():
    display(Markdown("###" + content.role + ":"))
    for part in content.parts:
        if part.text:
            display(Markdown(part.text))
        if part.function_call:
            print("Function call: {", part.function_call, "}")
        if part.function_response:
            print("Function response: {", part.function_response, "}")
    print("-" * 80)

Disco ball is spinning!
Starting music! energetic=True loud=True, bpm=130
Lights are now set to 30%


###user:

Turn this place into a party!

--------------------------------------------------------------------------------


###model:

...

###user:

Function response: { id=None name='power_disco_ball' response={'result': True} }
Function response: { id=None name='start_music' response={'result': 'Never gonna give you up.'} }
Function response: { id=None name='dim_lights' response={'result': True} }
--------------------------------------------------------------------------------


###model:

Alright, the party is starting! The disco ball is on, the lights are dimmed, and we've got some energetic music playing.

--------------------------------------------------------------------------------


As you can see, the model didn't wait for the different function calls and instead called multiple ones in parallel.

## Next Steps
### Useful API references:

- The [genai.Client](https://googleapis.github.io/python-genai/genai.html#module-genai.client) class
  - Its [Client.models.generate_content](https://googleapis.github.io/python-genai/genai.html#genai.models.Models.generate_content) method has a [genai.types.GenerateContentConfig](https://googleapis.github.io/python-genai/genai.html#genai.types.GenerateContentConfig) field that is in particular used to set the tools and function calls.
    - The config's `tools` attribute contains a list of [genai.types.Tool](https://googleapis.github.io/python-genai/genai.html#genai.types.Tool)s objects.
    - The `function_declarations` attribute contains a list of [genai.types.FunctionDeclarations](https://googleapis.github.io/python-genai/genai.html#genai.types.FunctionDeclaration) objects.
- The [response](https://googleapis.github.io/python-genai/genai.html#genai.types.GenerateContentResponse)'s [candidate](https://googleapis.github.io/python-genai/genai.html#genai.types.Candidate)'s [content](https://googleapis.github.io/python-genai/genai.html#genai.types.Content)'s [parts](https://googleapis.github.io/python-genai/genai.html#genai.types.Part) may contain a [genai.types.FunctionCall](https://googleapis.github.io/python-genai/genai.html#genai.types.FunctionCall), in `response.candidates[0].contents.parts[0]`.
- if `automatic_function_calling` is not disabled, the [genai.Chats](https://googleapis.github.io/python-genai/genai.html#module-genai.chats) session executes the call, and sends back the [genai.types.FunctionResponse]https://googleapis.github.io/python-genai/genai.html#genai.types.FunctionResponse).
- In response to a [FunctionCall](https://googleapis.github.io/python-genai/genai.html#genai.types.FunctionCall) the model always expects a [FunctionResponse](https://googleapis.github.io/python-genai/genai.html#genai.types.FunctionResponse).
- If you reply manually using [Chats.send_message](https://googleapis.github.io/python-genai/genai.html#genai.chats.AsyncChat.send_message) or [models.generate_content](https://googleapis.github.io/python-genai/genai.html#genai.models.Models.generate_content) remember thart the API is stateless you have to send the whole conversation history (a list of [Content](https://googleapis.github.io/python-genai/genai.html#genai.types.Content) objects), not just the last one containing the `FunctionResponse`.

### Related examples

Check those examples using function calling to give you more ideas on how to use that very useful feature:
* [Barista Bot](../examples/Agents_Function_Calling_Barista_Bot.ipynb), an agent to order coffee
* [Browser-as-a-tool](../examples/Browser_as_a_tool.ipynb), using function calling to call a web-browser.
* Using function calling to [re-rank seach results](../examples/Search_reranking_using_embeddings.ipynb).
* [Using tools with the Live API](../quickstarts/Get_started_LiveAPI_tools.ipynb), using function calling and other tools with the Live APIs.

### Continue your discovery of the Gemini API

Learn how to control how the Gemini API interact with your functions in the [function calling config](../quickstarts/Function_calling_config.ipynb) quickstart, discover how to control the model output in [JSON](../quickstarts/JSON_mode.ipynb) or using an [Enum](../quickstarts/Enum.ipynb) or learn how the Gemini API can generate and run code by itself using [Code execution](../quickstarts/Code_Execution.ipynb)